In [10]:
from xml.dom.minidom import (parse, Element)

In [11]:
corpus_filename = 'Tselina'
#corpus_filename = 'pedagogika'
#corpus_filename = 'sirija'

file_ext = '.xml'
doc = parse(corpus_filename + file_ext)
sentences = doc.getElementsByTagName('S') # все элементы с тегом S (все предложения корпуса)

In [12]:
f'Количество предложений в корпусе = {len(sentences)}'

'Количество предложений в корпусе = 189'

In [130]:
class Sentence:
    def __init__(self, sent: Element):
        self.id = sent.getAttribute('ID')
        self.words = []
        
        self.parseWords(sent.getElementsByTagName('W'))
        # получаем все элементы xml с тегом W (слова)
        
    def parseWords(self, rawWords):
        
        self.wordMap = {} # для последующего связывания слов
        # ключ - доминатор(айди доминатора) этого слова
        # значение - список слов, для которых доминатор равен ключу
        
        for rawWord in rawWords:
            
            word = Word(rawWord, self.id) # переносим в класс
            
            self.words.append(word) # добавляем в общий список слов
            
            if word.dom == '_root':
                self.rootWord = word # корневое слово
                
            # добавление в мапу
            if word.dom in self.wordMap.keys():
                self.wordMap[word.dom].append(word)
            else:
                self.wordMap[word.dom] = [word]

        # цикл по всем словам, чтобы связать их с помощью мапы
        for word in self.words:
            if word.id in self.wordMap.keys():
                for d in self.wordMap[word.id]:
                    word.addWord(d)
    def printGram(self):
        # печать грамматики предложения = печать грамматики одного слова(которое печатает все свои связанные слова, то есть все слова в предложении)
        self.rootWord.printGram(True) 
        
        
class Word:
    def __init__(self, w: Element, sentId):
        self.dom = w.getAttribute('DOM')
        self.feat = w.getAttribute('FEAT')
        self.id = w.getAttribute('ID')
        self.lemma = w.getAttribute('LEMMA')
        self.link = w.getAttribute('LINK')
        self.sentId = sentId
        
        self.connectedWords = [] #связанные с этим словом слова
        
    def addWord(self, w):
        # добавление слова в связанные с этим словом слова
        self.connectedWords.append(w)
        
    def printGram(self, checkProj):
        if checkProj:
            t = checkProjectivity(self, {}, 0)
            if not t:
                # закомментировать return, чтобы такие предложения не пропускались
                #print(f'\nNOT PROJECTIVE, {self.sentId}\n')
                return
        print(f'F([{self.feat}])->', end = '') # F(t) ->
        
        if self.connectedWords == []:
            print(f'[{self.feat}]\n', end = '') # когда узел - терминальный
            
        else:
            t = False # для расположения граммем слова в нужном месте
            for w in self.connectedWords:
                
                # проход по всем словам в связанных словах
                if not t and (int(w.id) > int(self.id)): 
                    # когда этого не происходило ранее и когда айди текущего связанного слова превысило айди самого слова
                    print(f'[{self.feat}]', end = '')
                    t = True
                    
                print(f';D([{w.feat}], {w.link});', end = '') # D(t, s)
            if not t:
                print(f'[{self.feat}]', end = '')
            for w in self.connectedWords:
                print(f'\nD([{w.feat}], {w.link})->F([{w.feat}])') #D(t, s) -> F(t)
                #print() # раскомментировать и закомментировать предыдущую, если не хотим печатать D(t, s) -> F(t)
                w.printGram(False)
                
def checkProjectivity(rootWord, seenWordsIds, floor):
    if floor not in seenWordsIds.keys():
        # создаем список для этого яруса
        seenWordsIds[floor] = []
    if any(map(lambda x: int(x) > int(rootWord.id), seenWordsIds[floor])):
        # если в истории этого яруса есть id больший, чем id текущего узла
        return False
    seenWordsIds[floor].append(rootWord.id) # добавляем в историю id текущего узла
    if rootWord.connectedWords == []: # если нет поддеревьев
        return True
    t = True
    for w in rootWord.connectedWords:
        # проверяем все поддеревья
        t = t and checkProjectivity(w, seenWordsIds, floor + 1)
        if not t:
            # если случай найден, то возвращаем сразу же
            return t
    return t   
        

In [112]:
def parseSentences():
    for sent in sentences:
        #пробегаемся по всем предложениям корпуса
        parseSentence(sent)

def parseSentence(sent: Element):
    sentence = Sentence(sent)
    sentence.printGram()

In [15]:
import sys
filename = corpus_filename + '_' + 'gram.out'

In [116]:
orig_stdout = sys.stdout
f = open(filename, 'w', encoding = 'utf-16')
sys.stdout = f
# перенаправление stdout в файл, чтобы все печаталось в файл

parseSentences()

# перенаправление обратно
sys.stdout = orig_stdout
f.close()

In [227]:
filename_sorted = corpus_filename + '_' + 'gram_sorted.out'

In [151]:
# сортировка и удаление дубликатов в файле
def printSortedNoDuplicatesFile():
    lines_seen = set()
    with open(filename, 'r', encoding = 'utf-16') as r:
        with open(filename_sorted, 'w', encoding = 'utf-16') as f:
            for line in sorted(r):
                if line not in lines_seen:
                    line = str(line.strip())

                    if len(line) != 0 and line[-1] == ';':
                        line = line[:-1].strip()
                    #if '||' in line:
                    line = line.replace(';;', ';')
                    line = line.replace('->;', '->')
                    line += '\n'  
                    if line == '\n':
                        continue
                    print(line, end = '', file = f)
                    lines_seen.add(line)

In [229]:
printSortedNoDuplicatesFile()

In [152]:
# для теста
# печатает грамматику одного предложения

sentence_id = 1 #id предложения в корпусе <S ID = 'sentence_id'>

filename = 'test/test.out'
filename_sorted = 'test/test_sorted.out'

orig_stdout = sys.stdout
f = open(filename, 'w', encoding = 'utf-16')
sys.stdout = f
# перенаправление stdout в файл, чтобы все печаталось в файл

sentence = Sentence(sentences[sentence_id - 1])
sentence.printGram()

# перенаправление обратно
sys.stdout = orig_stdout
f.close()

printSortedNoDuplicatesFile()

In [207]:
d_gram_map = {}
f_gram_map = {}

In [208]:
def addToMap(mapName, key, value):
    if key in mapName.keys():
        mapName[key].append(value)
    else:
        mapName[key] = [value]

def mapGrammar(filename):
    with open(filename, 'r', encoding = 'utf-16') as r:
        for line in r:
            s = line.split('->')
            p = s[0].strip()
            w = s[1].strip().replace('\n', '')
            if (p[0] == 'D'):
                addToMap(d_gram_map, p, w)
            else:
                addToMap(f_gram_map, p, w)

In [359]:
def printGrammarMap(mapName):
    for key, value in mapName.items():
        s = ' | '.join(value)
        s = ' '.join(s.split(';'))
        print(key, '->', s)
    print()

In [210]:
def dMapFirstStep():
    for key, value in d_gram_map.items():
        d_gram_map[key] = f_gram_map[d_gram_map[key][0]]

In [351]:

def removeRecursion(d_gram_map):
    new_map = {}
    t = True
    while t:
        t = False
        for key, value in d_gram_map.items():
            if 'D(' not in str(value) and key not in new_map.keys():
                t = True
                new_map[key] = value
                for k, v in d_gram_map.items():
                    l = []
                    if key in str(v):
                        for item in v:
                            for val in value:
                                l.append(item.replace(key, val))
                    if l != []:
                        d_gram_map[k] = l
                        
    return d_gram_map
            

In [350]:
import copy

In [366]:
d_gram_map = {}
f_gram_map = {}

mapGrammar('test/test_sorted.out')
printGrammarMap(d_gram_map | f_gram_map)
dMapFirstStep()
printGrammarMap(d_gram_map)
new_map = removeRecursion(copy.deepcopy(d_gram_map))

printGrammarMap(new_map)

D([A ЕД МУЖ ДАТ], опред) -> F([A ЕД МУЖ ДАТ])
D([A МН ВИН НЕОД], опред) -> F([A МН ВИН НЕОД])
D([A МН РОД], опред) -> F([A МН РОД])
D([ADV], обст) -> F([ADV])
D([CONJ], сочин) -> F([CONJ])
D([PR], 1-компл) -> F([PR])
D([PR], 2-компл) -> F([PR])
D([PR], обст) -> F([PR])
D([S ЕД ЖЕН ВИН НЕОД], предл) -> F([S ЕД ЖЕН ВИН НЕОД])
D([S ЕД МУЖ ДАТ НЕОД], предл) -> F([S ЕД МУЖ ДАТ НЕОД])
D([S ЕД СРЕД ВИН НЕОД], предл) -> F([S ЕД СРЕД ВИН НЕОД])
D([S МН ЖЕН РОД НЕОД], 1-компл) -> F([S МН ЖЕН РОД НЕОД])
D([S МН МУЖ ВИН НЕОД], 1-компл) -> F([S МН МУЖ ВИН НЕОД])
D([V СОВ ИЗЪЯВ ПРОШ ЕД МУЖ], соч-союзн) -> F([V СОВ ИЗЪЯВ ПРОШ ЕД МУЖ])
F([A ЕД МУЖ ДАТ]) -> [A ЕД МУЖ ДАТ]
F([A МН ВИН НЕОД]) -> [A МН ВИН НЕОД]
F([A МН РОД]) -> [A МН РОД]
F([ADV]) -> [ADV]
F([CONJ]) -> [CONJ] D([V СОВ ИЗЪЯВ ПРОШ ЕД МУЖ], соч-союзн)
F([PR]) -> [PR] D([S ЕД ЖЕН ВИН НЕОД], предл) | [PR] D([S ЕД МУЖ ДАТ НЕОД], предл) | [PR] D([S ЕД СРЕД ВИН НЕОД], предл)
F([S ЕД ЖЕН ВИН НЕОД]) -> [S ЕД ЖЕН ВИН НЕОД] D([PR], 2-компл)
F([S ЕД 